# 📊 Preprocesado de Datos - Proyectos Horizonte Europa

Este notebook se encarga de la limpieza y preparación de datos administrativos de proyectos europeos del Programa Marco 9 (Horizonte Europa).

## ⚠️ Consideraciones Especiales:
- **Datos administrativos**: NO eliminar valores nulos (son naturales en gestión)
- **Duplicados**: Solo por `Ref.CSIC` (clave primaria)
- **Variables string**: Muchas columnas numéricas son en realidad etiquetas (ej: `Ref.UE`, `Centro`)
- **Normalización**: `Nombre centro IP` debe normalizarse usando `Centro` como referencia

## Contenido:
1. Carga y exploración inicial
2. Identificación de tipos de variables
3. Limpieza de duplicados (Ref.CSIC)
4. Normalización de nombres de centros
5. Conversión de tipos de datos
6. Procesamiento de fechas
7. Análisis de valores nulos (sin eliminar)
8. Exportación de datos limpios

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 1. Carga de Datos

In [2]:
# Cargar datos desde Excel
df = pd.read_excel('../data/9PM_bootcamp.xlsx')

print(f"📁 Datos cargados: {df.shape[0]} filas (proyectos) y {df.shape[1]} columnas")
print(f"\n📋 Columnas encontradas:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

📁 Datos cargados: 719 filas (proyectos) y 25 columnas

📋 Columnas encontradas:
  1. Ref.CSIC
  2. Ref.UE
  3. Situación
  4. Tipo
  5. Programa
  6. Acción clave
  7. Tit.Acc.clave
  8. Título
  9. Comienzo
  10. Final
  11. Duración (meses)
  12. Concedido
  13. CSIC
  14. España (no CSIC)
  15. Total (Csic, Esp. y otros)
  16. Cód.área
  17. Acrónimo del proyecto
  18. Coordinador CSIC
  19. Convocatoria
  20. Nombre IP
  21. Centro
  22. Nombre Centro IP
  23. Resumen
  24. Keywords
  25. Lump Sum


## 2. Clasificación de Variables

Según la naturaleza de los datos administrativos de proyectos europeos

In [3]:
# Definir clasificación de variables según su función
variables_config = {
    'identificadores': ['Ref.CSIC', 'Ref.UE', 'Centro', 'Acrónimo del proyecto'],
    'categoricas': ['situación', 'programa', 'Acción Clave', 'Coordinador CSIC', 'nombre centro IP'],
    'busqueda': ['tipo', 'Tit.Acc.clave', 'Título', 'convocatoria', 'Nombre IP', 'Resumen', 'Keywords'],
    'numericas': ['Duración(meses)', 'Concedido', 'CSIC', 'España (no CSIC)', 'total (Csic, Esp. y otros)'],
    'temporales': ['Comienzo', 'Final'],
    'ignorar': ['Cod.area', 'lump_sum']
}

print("📊 CLASIFICACIÓN DE VARIABLES:")
print("=" * 70)
for tipo, cols in variables_config.items():
    print(f"\n🔹 {tipo.upper()}:")
    for col in cols:
        if col in df.columns:
            print(f"   ✓ {col}")
        else:
            print(f"   ✗ {col} (no encontrada)")

📊 CLASIFICACIÓN DE VARIABLES:

🔹 IDENTIFICADORES:
   ✓ Ref.CSIC
   ✓ Ref.UE
   ✓ Centro
   ✓ Acrónimo del proyecto

🔹 CATEGORICAS:
   ✗ situación (no encontrada)
   ✗ programa (no encontrada)
   ✗ Acción Clave (no encontrada)
   ✓ Coordinador CSIC
   ✗ nombre centro IP (no encontrada)

🔹 BUSQUEDA:
   ✗ tipo (no encontrada)
   ✓ Tit.Acc.clave
   ✓ Título
   ✗ convocatoria (no encontrada)
   ✓ Nombre IP
   ✓ Resumen
   ✓ Keywords

🔹 NUMERICAS:
   ✗ Duración(meses) (no encontrada)
   ✓ Concedido
   ✓ CSIC
   ✓ España (no CSIC)
   ✗ total (Csic, Esp. y otros) (no encontrada)

🔹 TEMPORALES:
   ✓ Comienzo
   ✓ Final

🔹 IGNORAR:
   ✗ Cod.area (no encontrada)
   ✗ lump_sum (no encontrada)


In [5]:
# Vista previa de datos
print("🔍 PRIMERAS FILAS DEL DATASET:")
print("=" * 70)
display(df.head())

print("\n📊 INFORMACIÓN BÁSICA:")
print(f"Total proyectos: {len(df)}")

🔍 PRIMERAS FILAS DEL DATASET:


,Ref.CSIC,Ref.UE,Situación,Tipo,Programa,Acción clave,Tit.Acc.clave,Título,Comienzo,Final,...,Cód.área,Acrónimo del proyecto,Coordinador CSIC,Convocatoria,Nombre IP,Centro,Nombre Centro IP,Resumen,Keywords,Lump Sum
0,EUROHPC/1238,101196247,VIGENTE,HORIZON-RIA,JOINT UNDERTAKING,JOINT UNDERTAKING,JOINT UNDERTAKING,Europe-India Partnership for Scientific High-P...,01/02/2025,31/01/2028,...,8903,GANANA,NO,HORIZON-EUROHPC-JU-2023-INCO-06-01,"FOLCH DURAN, ARNAU",30102.0,GEOCIENCIAS BARCELONA,HIGH-PERFORMANCE COMPUTING (HPC) IS A MAJOR EN...,NaN,0
1,HE/CBE/0653,101214822,VIGENTE,HORIZON-JU-IA,JOINT UNDERTAKING,JOINT UNDERTAKING,JOINT UNDERTAKING,Bio-based in Soil applications with Optimal bi...,01/09/2025,31/08/2029,...,8908,SOUL,NO,HORIZON-JU-CBE-2024,"ALCALDE GALEOTE, MIGUEL",20401.0,INSTO. CATALISIS Y PETROLEOQUIMICA,THE USE OF PLASTIC PRODUCTS HAS INCREASED SIGN...,IMPROVED CONTROLLED BIODEGRADABILIY; IMPROVED ...,0
2,HE/CBE/0653,101214822,VIGENTE,HORIZON-JU-IA,JOINT UNDERTAKING,JOINT UNDERTAKING,JOINT UNDERTAKING,Bio-based in Soil applications with Optimal bi...,01/09/2025,31/08/2029,...,8908,SOUL,NO,HORIZON-JU-CBE-2024,"ALCALDE GALEOTE, MIGUEL",20401.0,INSTO. CATALISIS Y PETROLEOQUIMICA,THE USE OF PLASTIC PRODUCTS HAS INCREASED SIGN...,IMPROVED CONTROLLED BIODEGRADABILIY; IMPROVED ...,0
3,HE/CLEANH2/0431,101137792,VIGENTE,OBS_HORIZON-IA,JTI-CLEANH2,JTI-CLEANH2,JTI-CLEANH2,A novel multi-stage steam gasification and syn...,01/01/2024,31/12/2027,...,8908,HYIELD,NO,HORIZON-JTI-CLEANH2-2023-1,"MURILLO VILLUENDAS, RAMON",20403.0,INSTO. CARBOQUIMICA,EUROPE FACES THE JOINT CHALLENGE OF DECARBONIS...,NaN,0
4,HE/CLEANH2/0571,101137756,VIGENTE,OBS_HORIZON-IA,JTI-CLEANH2,JTI-CLEANH2,JTI-CLEANH2,Carbon-negative pressurized hydrogen productio...,01/10/2024,30/09/2028,...,8908,CARMA-H2,NO,HORIZON-JTI-CLEANH2-2023-1,"SERRA ALFARO, JOSE MANUEL",20166.0,INSTO. TECNOLOGIA QUIMICA,CARMA-H2 WILL ENABLE HIGHLY ATTRACTIVE HYDROGE...,CARBON NEGATIVE ELECTROCHEMICAL HYDROGEN PRODU...,0



📊 INFORMACIÓN BÁSICA:
Total proyectos: 719


In [6]:
# Información detallada del dataset
print("📋 INFORMACIÓN DEL DATASET:")
print("=" * 70)
df.info()

📋 INFORMACIÓN DEL DATASET:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Ref.CSIC                    719 non-null    object 
 1   Ref.UE                      717 non-null    object 
 2   Situación                   719 non-null    object 
 3   Tipo                        719 non-null    object 
 4   Programa                    716 non-null    object 
 5   Acción clave                716 non-null    object 
 6   Tit.Acc.clave               716 non-null    object 
 7   Título                      719 non-null    object 
 8   Comienzo                    712 non-null    object 
 9   Final                       712 non-null    object 
 10  Duración (meses)            714 non-null    float64
 11  Concedido                   719 non-null    float64
 12  CSIC                        717 non-null    float64
 13  España (

## 3. Detección y Eliminación de Duplicados

**CRÍTICO**: Solo eliminar duplicados por `Ref.CSIC` (clave primaria)

In [7]:
# Verificar duplicados completos (informativos, no eliminar necesariamente)
duplicados_completos = df.duplicated().sum()
print(f"📊 Filas completamente duplicadas: {duplicados_completos}")

if duplicados_completos > 0:
    print("\n⚠️ Nota: Duplicados completos encontrados, pero pueden ser válidos en datos administrativos")

📊 Filas completamente duplicadas: 1

⚠️ Nota: Duplicados completos encontrados, pero pueden ser válidos en datos administrativos


## 4. Normalización de Nombres de Centros

**CRÍTICO**: `nombre centro IP` puede tener variaciones (abreviaturas).  
Normalizar usando `Centro` (código) como referencia.

In [ ]:
# Explorar relación Centro - nombre centro IP
if 'Centro' in df.columns and 'nombre centro IP' in df.columns:
    print("🔍 ANÁLISIS DE NOMBRES DE CENTROS:")
    print("=" * 70)
    
    # Agrupar por código de centro y ver variaciones de nombres
    centro_nombres = df.groupby('Centro')['nombre centro IP'].unique()
    
    # Detectar centros con múltiples nombres
    centros_multiples = {centro: nombres for centro, nombres in centro_nombres.items() 
                         if len(nombres) > 1}
    
    if centros_multiples:
        print(f"\n⚠️ Se encontraron {len(centros_multiples)} centros con múltiples variaciones de nombre:")
        for centro, nombres in list(centros_multiples.items())[:10]:  # Mostrar primeros 10
            print(f"\n  Centro código: {centro}")
            for nombre in nombres:
                print(f"    - {nombre}")
    else:
        print("✅ No se encontraron variaciones en nombres de centros")

In [ ]:
# Normalizar nombres de centros usando el nombre más frecuente por código
if 'Centro' in df.columns and 'nombre centro IP' in df.columns:
    # Crear mapeo: código centro → nombre más frecuente
    centro_nombre_map = {}
    
    for centro in df['Centro'].unique():
        if pd.notna(centro):
            # Obtener nombre más frecuente para este centro
            nombres = df[df['Centro'] == centro]['nombre centro IP']
            nombre_frecuente = nombres.mode()[0] if len(nombres.mode()) > 0 else nombres.iloc[0]
            centro_nombre_map[centro] = nombre_frecuente
    
    # Aplicar normalización
    df['nombre centro IP normalizado'] = df['Centro'].map(centro_nombre_map)
    
    # Contar cambios
    cambios = (df['nombre centro IP'] != df['nombre centro IP normalizado']).sum()
    print(f"\n✅ Normalización completada:")
    print(f"   Total de registros con nombre normalizado: {cambios}")
    print(f"   Centros únicos: {df['nombre centro IP normalizado'].nunique()}")
    
    # Mostrar ejemplo de normalización
    if cambios > 0:
        print("\n📊 Ejemplos de normalización:")
        ejemplos = df[df['nombre centro IP'] != df['nombre centro IP normalizado']][
            ['Centro', 'nombre centro IP', 'nombre centro IP normalizado']
        ].head(5)
        display(ejemplos)

## 5. Conversión de Tipos de Datos

Asegurar que las variables tengan el tipo correcto

In [ ]:
# Convertir Ref.UE y Centro a string (NO son numéricas)
columnas_string = ['Ref.UE', 'Centro']

for col in columnas_string:
    if col in df.columns:
        df[col] = df[col].astype(str)
        print(f"✅ {col} convertido a string")

# Convertir fechas
columnas_fecha = ['Comienzo', 'Final']

for col in columnas_fecha:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
        print(f"✅ {col} convertido a datetime")

# Asegurar tipos numéricos correctos
if 'Duración(meses)' in df.columns:
    df['Duración(meses)'] = pd.to_numeric(df['Duración(meses)'], errors='coerce')
    
if 'Concedido' in df.columns:
    df['Concedido'] = pd.to_numeric(df['Concedido'], errors='coerce')

for col in ['CSIC', 'España (no CSIC)', 'total (Csic, Esp. y otros)']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

print("\n✅ Conversión de tipos completada")

## 6. Limpieza de Strings

Eliminar espacios y normalizar formato

In [ ]:
# Limpiar columnas de texto
text_columns = df.select_dtypes(include=['object']).columns

for col in text_columns:
    if col in df.columns and df[col].dtype == 'object':
        # Eliminar espacios al inicio y final
        df[col] = df[col].str.strip()
        # Reemplazar múltiples espacios por uno solo
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True)

print(f"✅ Limpieza de {len(text_columns)} columnas de texto completada")

## 7. Análisis de Valores Nulos

**IMPORTANTE**: NO eliminar nulos (son naturales en datos administrativos)

In [ ]:
# Análisis de valores nulos (solo informativo)
import matplotlib.pyplot as plt
import seaborn as sns

null_counts = df.isnull().sum()
null_percentages = (null_counts / len(df)) * 100

null_info = pd.DataFrame({
    'Columna': df.columns,
    'Nulos': null_counts.values,
    'Porcentaje': null_percentages.values
}).sort_values(by='Nulos', ascending=False)

print("📊 ANÁLISIS DE VALORES NULOS:")
print("=" * 70)
print(null_info[null_info['Nulos'] > 0])

print("\n⚠️ NOTA: Los valores nulos NO se eliminan en datos administrativos")
print("   Son parte natural de la información y deben preservarse")

In [ ]:
# Visualización de nulos
plt.figure(figsize=(14, 6))
sns.heatmap(df.isnull(), cbar=True, cmap='YlOrRd', yticklabels=False)
plt.title('Mapa de Valores Nulos en el Dataset', fontsize=14, pad=15)
plt.xlabel('Variables')
plt.tight_layout()
plt.show()

## 8. Creación de Variables Derivadas

Extraer información adicional útil para análisis

In [ ]:
# Extraer información temporal
if 'Comienzo' in df.columns and pd.api.types.is_datetime64_any_dtype(df['Comienzo']):
    df['Año Inicio'] = df['Comienzo'].dt.year
    df['Mes Inicio'] = df['Comienzo'].dt.month
    df['Año Inicio (string)'] = df['Año Inicio'].astype(str)
    print("✅ Variables temporales creadas (Año Inicio, Mes Inicio)")

if 'Final' in df.columns and pd.api.types.is_datetime64_any_dtype(df['Final']):
    df['Año Fin'] = df['Final'].dt.year
    print("✅ Año Fin creado")

# Calcular presupuesto por mes
if 'Concedido' in df.columns and 'Duración(meses)' in df.columns:
    df['Presupuesto Mensual'] = df.apply(
        lambda row: row['Concedido'] / row['Duración(meses)'] 
        if pd.notna(row['Duración(meses)']) and row['Duración(meses)'] > 0 
        else np.nan, 
        axis=1
    )
    print("✅ Presupuesto Mensual calculado")

# Categorizar duración de proyectos
if 'Duración(meses)' in df.columns:
    def categorizar_duracion(meses):
        if pd.isna(meses):
            return 'Sin especificar'
        elif meses <= 12:
            return 'Corto (≤12 meses)'
        elif meses <= 36:
            return 'Medio (13-36 meses)'
        else:
            return 'Largo (>36 meses)'
    
    df['Categoría Duración'] = df['Duración(meses)'].apply(categorizar_duracion)
    print("✅ Categoría Duración creada")

# Categorizar presupuesto
if 'Concedido' in df.columns:
    def categorizar_presupuesto(cantidad):
        if pd.isna(cantidad):
            return 'Sin especificar'
        elif cantidad < 100000:
            return 'Pequeño (<100K)'
        elif cantidad < 500000:
            return 'Medio (100K-500K)'
        elif cantidad < 1000000:
            return 'Grande (500K-1M)'
        else:
            return 'Muy Grande (>1M)'
    
    df['Categoría Presupuesto'] = df['Concedido'].apply(categorizar_presupuesto)
    print("✅ Categoría Presupuesto creada")

print("\n✅ Todas las variables derivadas creadas exitosamente")

## 9. Resumen Final del Preprocesado

In [ ]:
# Resumen completo del dataset preprocesado
print("=" * 80)
print("📊 RESUMEN FINAL DEL DATASET PREPROCESADO")
print("=" * 80)

print(f"\n📁 DIMENSIONES:")
print(f"   Total de proyectos: {df.shape[0]:,}")
print(f"   Total de variables: {df.shape[1]}")

print(f"\n📅 PERÍODO TEMPORAL:")
if 'Comienzo' in df.columns:
    print(f"   Primer proyecto: {df['Comienzo'].min()}")
    print(f"   Último proyecto: {df['Final'].max() if 'Final' in df.columns else 'N/A'}")

print(f"\n💰 PRESUPUESTO:")
if 'Concedido' in df.columns:
    print(f"   Total concedido: {df['Concedido'].sum():,.2f} €")
    print(f"   Promedio por proyecto: {df['Concedido'].mean():,.2f} €")

print(f"\n🏛️ PARTICIPACIÓN:")
if 'CSIC' in df.columns:
    print(f"   Centros CSIC participantes: {df['CSIC'].sum()}")
if 'España (no CSIC)' in df.columns:
    print(f"   Centros españoles (no CSIC): {df['España (no CSIC)'].sum()}")

print(f"\n📊 PROGRAMAS:")
if 'programa' in df.columns:
    print(f"   Programas únicos: {df['programa'].nunique()}")
    print(f"   Top 3 programas:")
    for i, (prog, count) in enumerate(df['programa'].value_counts().head(3).items(), 1):
        print(f"      {i}. {prog}: {count} proyectos")

print(f"\n⚠️ VALORES NULOS:")
print(f"   Total: {df.isnull().sum().sum():,} valores nulos")
print(f"   Nota: Los nulos NO han sido eliminados (datos administrativos)")

print(f"\n✅ TRANSFORMACIONES APLICADAS:")
print(f"   ✓ Duplicados por Ref.CSIC eliminados")
print(f"   ✓ Nombres de centros normalizados")
print(f"   ✓ Tipos de datos convertidos correctamente")
print(f"   ✓ Variables derivadas creadas")

print("\n" + "=" * 80)

## 10. Exportación de Datos Limpios

In [ ]:
# Guardar dataset preprocesado
df.to_csv('../data/9PM_bootcamp_clean.csv', index=False, encoding='utf-8')
df.to_excel('../data/9PM_bootcamp_clean.xlsx', index=False)

print("✅ Datos exportados exitosamente:")
print("   📁 ../data/9PM_bootcamp_clean.csv")
print("   📁 ../data/9PM_bootcamp_clean.xlsx")
print(f"\n📊 Total de proyectos exportados: {len(df):,}")